# STEP 2: EfficientAD-M per Anomaly Detection - UN MODELLO PER OGNI CONNETTORE

Questo notebook addestra **9 modelli EfficientAD-M separati**, uno per ogni connettore (conn1, conn2, ..., conn9).

Ogni modello avrà il suo threshold specifico calcolato solo sui dati OK del rispettivo connettore.

**EfficientAD-M** è un metodo di anomaly detection basato su Teacher-Student architecture:
- **Teacher**: ResNet18 pre-addestrato su ImageNet (congelato)
- **Student**: ResNet18 non pre-addestrato (addestrato su OK)
- **Anomaly score**: differenza tra feature teacher e student

Funziona meglio degli autoencoder quando le anomalie sono strutturali/geometriche (come nel nostro caso).

**NOTA**: Le immagini sono già in grayscale e normalizzate nel preprocessing. Le carichiamo come RGB per compatibilità con ResNet pre-addestrato.


## Setup


In [ ]:
# Setup: Clona repository GitHub e monta Google Drive per i dati
import os
from pathlib import Path

# Opzione 1: Clona da GitHub (consigliato per sviluppo)
# Sostituisci con il tuo repository URL
GITHUB_REPO = "https://github.com/Giovanni000/Project-Work.git"  # ⚠️ MODIFICA QUESTO!
REPO_DIR = "/content/project"

# Clona repository (se non esiste già)
if not Path(REPO_DIR).exists():
    !git clone {GITHUB_REPO} {REPO_DIR}
else:
    os.chdir(REPO_DIR)
    !git pull

# Cambia directory al repository
os.chdir(REPO_DIR)
# Se il clone crea una sottocartella, entra dentro
subdirs = [d for d in Path(REPO_DIR).iterdir() if d.is_dir() and not d.name.startswith('.')]
if len(subdirs) == 1:
    os.chdir(subdirs[0])

print(f"Repository directory: {os.getcwd()}")

# Opzione 2: Monta Google Drive solo per i dati (immagini)
from google.colab import drive
drive.mount('/content/drive')

# Path ai dati su Drive
DATA_ROOT = Path("/content/drive/MyDrive/Project Work/Data")
print(f"Data directory: {DATA_ROOT}")

# ⚠️ IMPORTANTE: Le immagini su Drive sono LENTE da caricare durante il training!
# Se il training è troppo lento, considera di copiare le immagini in locale prima

# Import necessari
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights
from PIL import Image
import pandas as pd
from pathlib import Path
import numpy as np
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt

# Seed per riproducibilità
torch.manual_seed(42)
np.random.seed(42)

# Verifica device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")
    if "T4" in gpu_name:
        print("✅ Tesla T4 rilevata - Parametri ottimizzati per questa GPU")


## Verifica/Crea Dataset CSV


In [ ]:
# Se data/dataset.csv non esiste, crealo automaticamente
import pandas as pd
from pathlib import Path

dataset_csv = Path("data/dataset.csv")

if not dataset_csv.exists():
    print("⚠️  data/dataset.csv non trovato. Creazione automatica...")
    
    # Carica features_labeled.csv
    features_csv = Path("features_labeled.csv")
    if not features_csv.exists():
        # Prova path alternativo
        features_csv = Path("/content/project/features_labeled.csv")
    
    if not features_csv.exists():
        raise FileNotFoundError(f"features_labeled.csv non trovato in {features_csv}")
    
    print(f"Leggendo CSV: {features_csv}...")
    df = pd.read_csv(features_csv)
    print(f"CSV caricato: {len(df)} righe")
    
    # Aggrega PARTIAL OCCLUSION con OCCLUSION
    df['label_merged'] = df['label'].replace('PARTIAL OCCLUSION', 'OCCLUSION')
    
    # Costruisci path immagini (su Drive)
    DRIVE_DATA_BASE = "/content/drive/MyDrive/Project Work/Data"
    df['image_path'] = df.apply(
        lambda row: f"{DRIVE_DATA_BASE}/connectors/{row['connector_name']}/{row['filename']}",
        axis=1
    )
    
    # Verifica esistenza immagini
    print("Verificando esistenza immagini...")
    existing = []
    for idx, path in enumerate(df['image_path']):
        if Path(path).exists():
            existing.append(idx)
    
    print(f"Immagini trovate: {len(existing)}/{len(df)}")
    
    if len(existing) == 0:
        print("⚠️  PROBLEMA: Nessuna immagine trovata!")
        print("Verifica che Drive sia montato e che i path siano corretti.")
    
    # Filtra solo immagini esistenti
    df_valid = df.iloc[existing].copy()
    
    # Prepara CSV finale
    output_df = df_valid[['image_path', 'label_merged', 'connector_name']].copy()
    output_df.rename(columns={'label_merged': 'label'}, inplace=True)
    
    # Crea cartella data se non esiste
    dataset_csv.parent.mkdir(parents=True, exist_ok=True)
    
    # Salva
    output_df.to_csv(dataset_csv, index=False)
    print(f"✅ Dataset preparato: {len(output_df)} righe")
    print(f"Distribuzione label:")
    print(output_df['label'].value_counts())
else:
    print(f"✅ Dataset trovato: {dataset_csv}")
    df_check = pd.read_csv(dataset_csv)
    print(f"  Righe: {len(df_check)}")
    print(f"  Colonne: {list(df_check.columns)}")


## Configurazione EfficientAD-M


In [ ]:
# ============================================================================
# CONFIGURAZIONE EFFICIENTAD-M
# ============================================================================

# Dimensioni immagine (coerente con il progetto: 128x128)
IMG_SIZE = 128

# Parametri training
BATCH_SIZE = 32  # Ottimizzato per Tesla T4
NUM_EPOCHS = 20  # EfficientAD-M converge velocemente
LEARNING_RATE = 1e-4

# Threshold
THRESHOLD_MULTIPLIER = 2.5  # Threshold = mu + THRESHOLD_MULTIPLIER * sigma

# DataLoader
NUM_WORKERS = 2  # Ottimizzato per Tesla T4

# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Configurazione EfficientAD-M:")
print(f"  IMG_SIZE: {IMG_SIZE}")
print(f"  BATCH_SIZE: {BATCH_SIZE}")
print(f"  NUM_EPOCHS: {NUM_EPOCHS}")
print(f"  LEARNING_RATE: {LEARNING_RATE}")
print(f"  THRESHOLD_MULTIPLIER: {THRESHOLD_MULTIPLIER}")


## Dataset Class (solo OK, filtrato per connettore)


In [ ]:
class EfficientADDatasetPerConnector(Dataset):
    """
    Dataset PyTorch per EfficientAD-M di un singolo connettore.
    Contiene solo immagini OK del connettore specificato.
    
    NOTA: Le immagini sono già in grayscale e normalizzate nel preprocessing.
    Le carichiamo come RGB per compatibilità con ResNet pre-addestrato.
    """
    
    def __init__(self, csv_path, connector_name, transform=None):
        """
        Args:
            csv_path: Path al CSV con colonne 'image_path', 'label', 'connector_name'
            connector_name: Nome del connettore (es. 'conn1', 'conn2', ...)
            transform: Trasformazioni da applicare alle immagini
        """
        df = pd.read_csv(csv_path)
        # Filtra solo OK del connettore specificato
        self.df = df[(df['label'] == 'OK') & (df['connector_name'] == connector_name)].copy().reset_index(drop=True)
        self.transform = transform
        
        print(f"Dataset EfficientAD per {connector_name}: {len(self.df)} immagini OK")
        if len(self.df) == 0:
            raise ValueError(f"Nessuna immagine OK trovata per {connector_name}!")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        
        # Carica immagine (ottimizzato: evita lazy loading)
        try:
            # Le immagini sono già grayscale nel preprocessing, ma le carichiamo come RGB
            # per compatibilità con ResNet pre-addestrato
            image = Image.open(image_path).convert('RGB')
            image.load()  # Forza caricamento completo
        except Exception as e:
            print(f"Errore caricamento {image_path}: {e}")
            # Fallback: immagine nera
            image = Image.new('RGB', (IMG_SIZE, IMG_SIZE), (0, 0, 0))
        
        if self.transform:
            image = self.transform(image)
        
        return image


## Modelli Teacher & Student


In [ ]:
class Teacher(nn.Module):
    """
    Teacher: ResNet18 pre-addestrato su ImageNet (congelato).
    Usato per estrarre feature di riferimento.
    """
    def __init__(self):
        super(Teacher, self).__init__()
        # Carica ResNet18 pre-addestrato
        base = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        # Rimuovi layer finali (avgpool e fc), mantieni solo encoder
        self.encoder = nn.Sequential(*list(base.children())[:-2])
        
        # Congela tutti i parametri (no training)
        for param in self.encoder.parameters():
            param.requires_grad = False
    
    def forward(self, x):
        """
        Args:
            x: Tensor [B, 3, H, W]
        Returns:
            feature_map: Tensor [B, 512, Hf, Wf] dove Hf=H/32, Wf=W/32
        """
        return self.encoder(x)


class Student(nn.Module):
    """
    Student: ResNet18 NON pre-addestrato (pesi random).
    Addestrato per imitare le feature del Teacher su immagini OK.
    """
    def __init__(self):
        super(Student, self).__init__()
        # Carica ResNet18 SENZA pesi pre-addestrati
        base = resnet18(weights=None)
        # Rimuovi layer finali (avgpool e fc), mantieni solo encoder
        self.encoder = nn.Sequential(*list(base.children())[:-2])
    
    def forward(self, x):
        """
        Args:
            x: Tensor [B, 3, H, W]
        Returns:
            feature_map: Tensor [B, 512, Hf, Wf] dove Hf=H/32, Wf=W/32
        """
        return self.encoder(x)


## Funzione Training per Connettore


In [ ]:
def train_efficientad_per_connector(connector_name, csv_path="data/dataset.csv",
                                     batch_size=32,
                                     num_epochs=20,
                                     learning_rate=1e-4,
                                     device=None):
    """
    Addestra un modello EfficientAD-M per un singolo connettore.
    
    Args:
        connector_name: Nome del connettore (es. 'conn1')
        csv_path: Path al CSV del dataset
        batch_size: Dimensione del batch
        num_epochs: Numero di epoche
        learning_rate: Learning rate
        device: Device (cuda/cpu)
    
    Returns:
        teacher: Modello Teacher (congelato)
        student: Modello Student addestrato
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print(f"\n{'='*60}")
    print(f"Training EfficientAD-M per {connector_name}")
    print(f"{'='*60}")
    
    # Trasformazioni (normalizzazione ImageNet per ResNet pre-addestrato)
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Dataset (solo OK del connettore specificato)
    dataset = EfficientADDatasetPerConnector(csv_path, connector_name, transform=transform)
    
    # DataLoader (ottimizzato per Colab)
    train_loader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=NUM_WORKERS,
        pin_memory=True if device.type == 'cuda' else False,
        prefetch_factor=2,
        persistent_workers=False
    )
    
    # Modelli
    teacher = Teacher().to(device)
    student = Student().to(device)
    
    # Teacher in modalità eval e congelato
    teacher.eval()
    for param in teacher.parameters():
        param.requires_grad = False
    
    # Loss e Optimizer (solo per Student)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)
    
    # Training loop
    print(f"\nTraining Student per imitare Teacher su immagini OK...")
    student.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images = images.to(device)
            
            # Feature Teacher (congelato, no grad)
            with torch.no_grad():
                teacher_features = teacher(images)
            
            # Feature Student (addestrato)
            student_features = student(images)
            
            # Loss: differenza tra feature Teacher e Student
            loss = criterion(student_features, teacher_features)
            
            # Backward solo su Student
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.6f}")
    
    # Salva modello Student
    models_dir = Path("models")
    models_dir.mkdir(exist_ok=True)
    model_path = models_dir / f"efficientad_student_{connector_name}.pth"
    torch.save(student.state_dict(), model_path)
    print(f"\n✅ Modello Student salvato in: {model_path}")
    
    return teacher, student


In [ ]:
def calculate_threshold_per_connector(teacher, student, connector_name, csv_path="data/dataset.csv", 
                                      threshold_multiplier=2.5, device=None):
    """
    Calcola il threshold per anomaly detection per un singolo connettore.
    
    Threshold = mu + threshold_multiplier * sigma, dove mu e sigma sono media e std
    degli anomaly score su tutte le immagini OK del connettore specificato.
    
    Anomaly score = differenza tra feature Teacher e Student (max su feature map).
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    print(f"\nCalcolo threshold per {connector_name}...")
    
    # Trasformazioni (stesse del training)
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Dataset (solo OK del connettore)
    dataset = EfficientADDatasetPerConnector(csv_path, connector_name, transform=transform)
    
    # DataLoader
    loader = DataLoader(
        dataset, 
        batch_size=1,  # Batch size 1 per calcolo preciso
        shuffle=False,
        num_workers=0,  # Evita problemi con multiprocessing
        pin_memory=False
    )
    
    # Calcola score per tutte le immagini OK
    scores = []
    
    teacher.eval()
    student.eval()
    
    with torch.no_grad():
        for images in tqdm(loader, desc=f"Calcolo score {connector_name}"):
            images = images.to(device)
            
            # Feature Teacher e Student
            t_feat = teacher(images)
            s_feat = student(images)
            
            # Differenza feature (anomaly map)
            diff = (t_feat - s_feat) ** 2
            # Media sui canali: [B, C, H, W] -> [B, H, W]
            amap = diff.mean(dim=1)
            # Score immagine = max su tutta la feature map
            score = amap.flatten(1).max(1)[0].cpu().item()
            scores.append(score)
    
    scores = np.array(scores)
    mu = np.mean(scores)
    sigma = np.std(scores)
    threshold = mu + threshold_multiplier * sigma
    
    print(f"  Score OK - mean: {mu:.6f}, std: {sigma:.6f}")
    print(f"  Threshold ({threshold_multiplier}*sigma): {threshold:.6f}")
    
    # Salva threshold
    models_dir = Path("models")
    threshold_path = models_dir / f"efficientad_threshold_{connector_name}.npy"
    np.save(threshold_path, threshold)
    print(f"  ✅ Threshold salvato in: {threshold_path}")
    
    return threshold


## Training Loop - Tutti i Connettori


In [ ]:
# Training per tutti i 9 connettori
connectors = [f"conn{i}" for i in range(1, 10)]

trained_models = {}

for connector_name in connectors:
    try:
        # Training
        teacher, student = train_efficientad_per_connector(
            connector_name=connector_name,
            csv_path="data/dataset.csv",
            batch_size=BATCH_SIZE,
            num_epochs=NUM_EPOCHS,
            learning_rate=LEARNING_RATE,
            device=DEVICE
        )
        
        trained_models[connector_name] = (teacher, student)
        
        # Calcolo threshold
        threshold = calculate_threshold_per_connector(
            teacher=teacher,
            student=student,
            connector_name=connector_name,
            csv_path="data/dataset.csv",
            threshold_multiplier=THRESHOLD_MULTIPLIER,
            device=DEVICE
        )
        
    except Exception as e:
        print(f"\n❌ Errore durante training di {connector_name}: {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*60}")
print(f"✅ Training completato per {len(trained_models)} connettori")
print(f"{'='*60}")


## Funzione di Caricamento Modello


In [ ]:
def load_efficientad_model(connector_name, device=None):
    """
    Carica un modello EfficientAD-M addestrato per un connettore.
    
    Args:
        connector_name: Nome del connettore (es. 'conn1')
        device: Device (cuda/cpu)
    
    Returns:
        teacher: Modello Teacher
        student: Modello Student
        threshold: Threshold per anomaly detection
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    models_dir = Path("models")
    model_path = models_dir / f"efficientad_student_{connector_name}.pth"
    threshold_path = models_dir / f"efficientad_threshold_{connector_name}.npy"
    
    if not model_path.exists():
        raise FileNotFoundError(f"Modello non trovato: {model_path}")
    
    if not threshold_path.exists():
        raise FileNotFoundError(f"Threshold non trovato: {threshold_path}")
    
    # Carica Teacher (sempre lo stesso, pre-addestrato)
    teacher = Teacher().to(device)
    teacher.eval()
    
    # Carica Student
    student = Student().to(device)
    student.load_state_dict(torch.load(model_path, map_location=device))
    student.eval()
    
    # Carica threshold
    threshold = np.load(threshold_path)
    
    print(f"✅ Modello EfficientAD-M caricato per {connector_name}")
    print(f"  Threshold: {threshold:.6f}")
    
    return teacher, student, threshold
